In [2]:
import pandas as pd

In [3]:
events = pd.read_csv('events.csv')
item_features = pd.read_csv('item_features.csv')
user_features = pd.read_csv('user_features.csv')

In [4]:
events

,user_id,item_id,rating,timestamp
0,0,1505,4,0
1,0,3669,3,1
2,0,584,4,2
3,0,3390,3,3
4,0,2885,4,4
...,...,...,...,...
894144,6039,1453,4,223
894145,6039,613,5,224
894146,6039,1548,4,225
894147,6039,241,4,226


In [5]:
user_features

,user_id,gender,age
0,4855,F,1
1,4065,M,56
2,3331,M,25
3,5373,M,45
4,2032,M,25
...,...,...,...
6035,4455,F,25
6036,5002,F,45
6037,4365,F,56
6038,3700,F,45


In [6]:
item_features

,item_id,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17
0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,3701,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3702,3702,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3703,3703,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3704,3704,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [7]:
# минимальное количество айтемов у пользователей
events.groupby(by='user_id').count()['item_id'].min()

14

In [8]:
# нагенерим фичи для юзеров
features = [
    # средний рейтинг за все время
    events.groupby(by='user_id').agg({'rating': 'mean'}).rename(columns={'rating': 'avg_rating'}),
    # количество всех айтемов
    events.groupby(by='user_id')[['item_id']].count().rename(columns={'item_id': 'cnt_item'})
]

# итоговая таблица
res_user_df = pd.concat(features, axis=1)

In [9]:
# средний рейтинг последних 3,5,7,10 айтемов
for n in [3,5,7,10]:
    tmp = events.groupby(by='user_id').tail(n).groupby(by='user_id')[['rating']].mean()\
                .rename(columns={'rating': f'avg_rating_last{n}'})
    res_user_df = pd.concat([res_user_df, tmp], axis=1)

In [10]:
# количество айтемов с рейтингами от 1 до 5 за все время
for rating in events.rating.unique():
    tmp = events[events['rating'] == rating].groupby(by='user_id')[['item_id']].count()\
                .rename(columns={'item_id': f'cnt_item_rating{rating}'})
    res_user_df = pd.merge(res_user_df, tmp, how='left', on='user_id').fillna(0)

In [11]:
# добавим фичи из таблицы user_features
res_user_df = pd.merge(res_user_df, user_features, how='inner', on='user_id').set_index('user_id')

In [12]:
res_user_df.to_csv('user_features.csv')

In [13]:
# нагенерим фичи для айтемов
# количество жанров в айтеме
res_item_df = pd.DataFrame(item_features.set_index('item_id').sum(axis=1)).rename(columns={0: 'cnt_genres'})



In [14]:
# нагенерим парные фичи из item_features и user_features
common_df = pd.merge(
    pd.merge(events, user_features, on='user_id', how='left'),
    item_features, 
    on='item_id', how='left'
)

In [15]:
# средний рейтинг по каждому жанру 
# и количество просмотренных фильмов по каждому жанру
for genre in item_features.columns[1:]:
    tmp = common_df[common_df[genre] == 1].groupby('user_id')[['rating']].mean()\
            .rename(columns={'rating': f'avg_{genre}_rating'})
    res_user_df = pd.merge(res_user_df, tmp, how='left', on='user_id').fillna(0)
    
    tmp = common_df[common_df[genre] == 1].groupby('user_id')[['item_id']].count()\
            .rename(columns={'item_id': f'cnt_{genre}_rating'})
    res_user_df = pd.merge(res_user_df, tmp, how='left', on='user_id').fillna(0)

In [16]:
# количество айтемов с рейтингами от 1 до 5 по каждому жанру
for genre in item_features.columns[1:]:
    for rating in events.rating.unique():
        tmp = common_df[(common_df[genre] == 1) & (common_df['rating'] == rating)].groupby('user_id')[['item_id']].count()\
                    .rename(columns={'item_id': f'cnt_{genre}_item_rating{rating}'})
        res_user_df = pd.merge(res_user_df, tmp, how='left', on='user_id').fillna(0)

In [22]:
res_item_df.to_csv("item_features_our.csv")
res_user_df.to_csv("user_features_our.csv")

In [18]:
# объединим все результаты в одну таблицу

res = pd.merge(events, res_user_df, how='left', on='user_id')
res = pd.merge(res, res_item_df, how='left', on='item_id')

In [19]:
res

,user_id,item_id,rating,timestamp,avg_rating,cnt_item,avg_rating_last3,avg_rating_last5,avg_rating_last7,avg_rating_last10,...,cnt_genre_16_item_rating3,cnt_genre_16_item_rating5,cnt_genre_16_item_rating2,cnt_genre_16_item_rating1,cnt_genre_17_item_rating4,cnt_genre_17_item_rating3,cnt_genre_17_item_rating5,cnt_genre_17_item_rating2,cnt_genre_17_item_rating1,cnt_genres
0,0,1505,4,0,3.979094,287,3.0,3.2,3.142857,3.4,...,2.0,9.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,3
1,0,3669,3,1,3.979094,287,3.0,3.2,3.142857,3.4,...,2.0,9.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,2
2,0,584,4,2,3.979094,287,3.0,3.2,3.142857,3.4,...,2.0,9.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,3
3,0,3390,3,3,3.979094,287,3.0,3.2,3.142857,3.4,...,2.0,9.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,2
4,0,2885,4,4,3.979094,287,3.0,3.2,3.142857,3.4,...,2.0,9.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894144,6039,1453,4,223,3.816832,202,4.0,4.2,4.285714,4.2,...,1.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1
894145,6039,613,5,224,3.816832,202,4.0,4.2,4.285714,4.2,...,1.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2
894146,6039,1548,4,225,3.816832,202,4.0,4.2,4.285714,4.2,...,1.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4
894147,6039,241,4,226,3.816832,202,4.0,4.2,4.285714,4.2,...,1.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2


In [20]:
res.to_csv('sample.csv')